### Парсинг HTML. XPath

#### Задание 1.
Написать приложение, которое собирает основные новости с сайтов news.mail.ru, lenta.ru, yandex.ru/news.
Для парсинга использовать xpath. Структура данных должна содержать:
название источника,
наименование новости,
ссылку на новость,
дата публикации.

In [1]:
from pprint import pprint
from lxml import html
import requests as rq
from pymongo import MongoClient
from datetime import date

In [2]:
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.123 Safari/537.36',
          'Accept':'*/*'}

In [3]:
def news_from_yandex():

    main_link_y = 'https://yandex.ru'
    sub_link_y = '/news/'
    
    response_y = rq.get(main_link_y + sub_link_y, headers=header)
    dom_y = html.fromstring(response_y.text)

    news_y = []
    
    data_y = dom_y.xpath("//div[@class='stories-set stories-set_main_no stories-set_pos_11']//td[@class='stories-set__item']")
    
    for d_y in data_y:
        
        item_y = {}
        
        source_y = d_y.xpath(".//div[@class='story__date']/text()")
        name_y = d_y.xpath(".//h2[@class='story__title']//text()")
        link_y = d_y.xpath(".//h2[@class='story__title']//@href")
        date_y = source_y[0].split()[-1]

        item_y['source'] = ''.join(source_y[0].split()[:-1]).replace('вчера', '').replace('в', '')
        item_y['name'] = name_y[0].replace('\xa0', ' ')
        item_y['link'] = main_link_y + link_y[0]
        item_y['date'] = f'{date.today()} {date_y}'
        
        news_y.append(item_y)
        
    return(news_y)

In [4]:
news_from_yandex()

[{'source': 'Ferra',
  'name': 'Робот-лаборант ускорит проведение исследований в тысячу раз',
  'link': 'https://yandex.ru/news/story/Robot-laborant_uskorit_provedenie_issledovanij_v_tysyachu_raz--608ec68a6c9b12451880896191ca0cc8?lr=2&lang=ru&stid=qpo4&persistent_id=105071253&rubric=science&from=index',
  'date': '2020-07-08 21:38'},
 {'source': 'Live24',
  'name': 'Жир на животе провоцирует ускоряющие старение заболевания',
  'link': 'https://yandex.ru/news/story/ZHir_na_zhivote_provociruet_uskoryayushhie_starenie_zabolevaniya--f01fdab8a6a8c019602955b103444fc9?lr=2&lang=ru&stid=hRFm4FZT4IldaOlf-ZM3&persistent_id=104932811&rubric=science&from=index',
  'date': '2020-07-08 21:56'},
 {'source': 'Evo-rus.com',
  'name': 'Разработан препарат, на клеточном уровне защищающий от радиации',
  'link': 'https://yandex.ru/news/story/Razrabotan_preparat_na_kletochnom_urovne_zashhishhayushhij_ot_radiacii--7d3793c3548b32b6de0d084913376704?lr=2&lang=ru&stid=PfM5OPLWtDPaT1aHvO6w&persistent_id=10499174

In [5]:
def news_from_lenta():
    
    main_link_l = 'https://lenta.ru'
    
    response_l = rq.get(main_link_l, headers=header)
    dom_l = html.fromstring(response_l.text)

    news_l = []
    
    data_l = dom_l.xpath("//section[@class='row b-top7-for-main js-top-seven']//div[@class='item']")
    
    for d_l in data_l:
        
        item_l = {}
        
        source_l = 'Lenta.ru'
        name_l = d_l.xpath(".//a/text()")
        link_l = d_l.xpath(".//a/@href")
        date_l = d_l.xpath(".//time[@class='g-time']/text()")

        item_l['source'] = source_l
        item_l['name'] = name_l[0].replace('\xa0', ' ')
        item_l['link'] = main_link_l + link_l[0]
        item_l['date'] = f'{date.today()} {date_l[0]}'
        
        news_l.append(item_l)
        
    return(news_l)

In [6]:
news_from_lenta()

[{'source': 'Lenta.ru',
  'name': 'В Минздраве усомнились в отсутствии иммунитета у переболевших COVID-19',
  'link': 'https://lenta.ru/news/2020/07/08/immune_covid/',
  'date': '2020-07-08 22:15'},
 {'source': 'Lenta.ru',
  'name': 'В российском регионе обнаружили примотанный скотчем к дереву труп мужчины',
  'link': 'https://lenta.ru/news/2020/07/08/primotali/',
  'date': '2020-07-08 22:05'},
 {'source': 'Lenta.ru',
  'name': 'Стали известны сроки запуска Spotify в России',
  'link': 'https://lenta.ru/news/2020/07/08/spotify/',
  'date': '2020-07-08 21:59'},
 {'source': 'Lenta.ru',
  'name': 'Модель с заболеванием кожи показала декольте и перелет в роскошном самолете',
  'link': 'https://lenta.ru/news/2020/07/08/winnieair/',
  'date': '2020-07-08 21:56'},
 {'source': 'Lenta.ru',
  'name': '«Манчестер Сити» забил пять безответных мячей середняку АПЛ',
  'link': 'https://lenta.ru/news/2020/07/08/mancityfootball/',
  'date': '2020-07-08 21:54'},
 {'source': 'Lenta.ru',
  'name': 'В Росс

In [7]:
def news_from_mail():
    
    main_link_m = 'https://news.mail.ru'
    
    response_m = rq.get(main_link_m, headers=header)
    dom_m = html.fromstring(response_m.text)

    news_m = []
    
    data_m = dom_m.xpath("//div[@class='block block_bg_primary block_separated_top link-hdr']//span[@class='list__text']")
    
    for d_m in data_m:
        
        item_m = {}
        
        name_m = d_m.xpath(".//span/text()")
        link_m = d_m.xpath("./a/@href")
        link_m = str(link_m[0])
        
        response_m_new = rq.get(link_m, headers=header)
        dom_m_new = html.fromstring(response_m_new.text)
        data_m_new = dom_m_new.xpath("//div[@class='breadcrumbs breadcrumbs_article js-ago-wrapper']")
        
        for d_m_new in data_m_new:
            source_m = d_m_new.xpath(".//span[@class='link__text']/text()")
            date_m = d_m_new.xpath(".//span[@class='note__text breadcrumbs__text js-ago']/text()")

        item_m['source'] = source_m[0].replace('\xa0', ' ')
        item_m['name'] = name_m[0].replace('\xa0', ' ')
        item_m['link'] = link_m
        item_m['date'] = f'{date.today()} {date_m[0]}'.replace(' (мск)', '')
        
        news_m.append(item_m)
        
    return(news_m)

In [8]:
news_from_mail()

[{'source': 'Коммерсантъ',
  'name': 'Дрозденко анонсировал изменения в «мусорной» реформе Ленобласти',
  'link': 'https://news.mail.ru/economics/42495731/',
  'date': '2020-07-08 15:16'},
 {'source': 'Piter.tv',
  'name': 'На корону «Миссис Россия 2020» претендуют две петербурженки',
  'link': 'https://news.mail.ru/society/42495677/',
  'date': '2020-07-08 15:15'},
 {'source': 'Piter.tv',
  'name': 'В Петербурге первую пятиэтажку расселят по программе реновации',
  'link': 'https://news.mail.ru/economics/42500455/',
  'date': '2020-07-08 21:22'},
 {'source': 'МК.RU Санкт-Петербург',
  'name': '17 пар Петербурга получили медали «За любовь и верность»',
  'link': 'https://news.mail.ru/society/42497191/',
  'date': '2020-07-08 16:12'}]

#### Задание 2.
Сложить все новости в БД.

In [9]:
client = MongoClient('localhost',27017)
db = client['news_db']
news = db.news

In [10]:
news_from_yandex = news_from_yandex()
news_from_lenta = news_from_lenta()
news_from_mail = news_from_mail()

In [11]:
def add_news_to_db(news_from_yandex, news_from_lenta, news_from_mail):
    
    news.insert_many(news_from_yandex)
    news.insert_many(news_from_lenta)
    news.insert_many(news_from_mail)

    return True

In [12]:
add_news_to_db(news_from_yandex, news_from_lenta, news_from_mail)

True